# Introduction

<center><img src="https://i.imgur.com/9hLRsjZ.jpg" height=400></center>

This dataset was scraped from [nextspaceflight.com](https://nextspaceflight.com/launches/past/?page=1) and includes all the space missions since the beginning of Space Race between the USA and the Soviet Union in 1957!

### Install Package with Country Codes

In [1]:
%pip install iso3166

Note: you may need to restart the kernel to use updated packages.


### Upgrade Plotly

Run the cell below if you are working with Google Colab.

In [2]:
%pip install --upgrade plotly

Requirement already up-to-date: plotly in /Users/Cedoula/opt/anaconda3/lib/python3.8/site-packages (4.14.3)
Note: you may need to restart the kernel to use updated packages.


### Import Statements

In [16]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# These might be helpful:
from iso3166 import countries
from datetime import datetime, timedelta

### Notebook Presentation

In [17]:
pd.options.display.float_format = '{:,.2f}'.format

### Load the Data

In [18]:
df_data = pd.read_csv('mission_launches.csv')

In [19]:
df_data.sample(5)

,Unnamed: 0,Unnamed: 0.1,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status
3048,3048,3048,RVSN USSR,"Site 43/4, Plesetsk Cosmodrome, Russia","Sat Apr 20, 1974 20:53 UTC",Molniya-M /Block ML | Molniya-1 n†­40,StatusRetired,NaN,Success
3647,3647,3647,General Dynamics,"LC-13, Cape Canaveral AFS, Florida, USA","Tue Aug 06, 1968 11:08 UTC",Atlas-SLV3A Agena-D | Canyon 1 (AFP-827),StatusRetired,NaN,Success
3257,3257,3257,US Air Force,"SLC-4W, Vandenberg AFB, California, USA","Fri Mar 17, 1972 17:00 UTC",Titan III(24)B | OPS 1678,StatusRetired,NaN,Success
4021,4021,4021,RVSN USSR,"Site 31/6, Baikonur Cosmodrome, Kazakhstan","Fri Aug 28, 1964 16:00 UTC",Vostok-2M | Cosmos 44,StatusRetired,NaN,Success
668,668,668,Land Launch,"Site 45/1, Baikonur Cosmodrome, Kazakhstan","Wed Oct 05, 2011 21:00 UTC",Zenit-3 SLB | Intelsat 18,StatusActive,NaN,Success


# Preliminary Data Exploration

* What is the shape of `df_data`? 
* How many rows and columns does it have?
* What are the column names?
* Are there any NaN values or duplicates?

In [20]:
print(f'The shape of the dataset is {df_data.shape}.')
print(f'The dataset has {df_data.shape[0]} rows and {df_data.shape[1]} columns.')
print('The column names are:')
for column in df_data.columns:
    print(column)

The shape of the dataset is (4324, 9).
The dataset has 4324 rows and 9 columns.
The column names are:
Unnamed: 0
Unnamed: 0.1
Organisation
Location
Date
Detail
Rocket_Status
Price
Mission_Status


In [21]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4324 entries, 0 to 4323
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      4324 non-null   int64 
 1   Unnamed: 0.1    4324 non-null   int64 
 2   Organisation    4324 non-null   object
 3   Location        4324 non-null   object
 4   Date            4324 non-null   object
 5   Detail          4324 non-null   object
 6   Rocket_Status   4324 non-null   object
 7   Price           964 non-null    object
 8   Mission_Status  4324 non-null   object
dtypes: int64(2), object(7)
memory usage: 304.2+ KB


## Data Cleaning - Check for Missing Values and Duplicates

Consider removing columns containing junk data. 

In [22]:
print(f'There are {df_data.isna().sum().sum()} NaN values in the dataset.')
print(f'There are {df_data.duplicated().sum().sum()} duplicates in the dataset.')

There are 3360 NaN values in the dataset.
There are 0 duplicates in the dataset.


In [23]:
df_data.drop(columns=['Unnamed: 0','Unnamed: 0.1'], inplace=True)
df_data.head()

,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status
0,SpaceX,"LC-39A, Kennedy Space Center, Florida, USA","Fri Aug 07, 2020 05:12 UTC",Falcon 9 Block 5 | Starlink V1 L9 & BlackSky,StatusActive,50.0,Success
1,CASC,"Site 9401 (SLS-2), Jiuquan Satellite Launch Ce...","Thu Aug 06, 2020 04:01 UTC",Long March 2D | Gaofen-9 04 & Q-SAT,StatusActive,29.75,Success
2,SpaceX,"Pad A, Boca Chica, Texas, USA","Tue Aug 04, 2020 23:57 UTC",Starship Prototype | 150 Meter Hop,StatusActive,NaN,Success
3,Roscosmos,"Site 200/39, Baikonur Cosmodrome, Kazakhstan","Thu Jul 30, 2020 21:25 UTC",Proton-M/Briz-M | Ekspress-80 & Ekspress-103,StatusActive,65.0,Success
4,ULA,"SLC-41, Cape Canaveral AFS, Florida, USA","Thu Jul 30, 2020 11:50 UTC",Atlas V 541 | Perseverance,StatusActive,145.0,Success


## Descriptive Statistics

In [24]:
df_data.describe()

,Organisation,Location,Date,Detail,Rocket_Status,Price,Mission_Status
count,4324,4324,4324,4324,4324,964,4324
unique,56,137,4319,4278,2,56,4
top,RVSN USSR,"Site 31/6, Baikonur Cosmodrome, Kazakhstan","Tue Jun 26, 1973",Cosmos-3MRB (65MRB) | BOR-5 Shuttle,StatusRetired,450.0,Success
freq,1777,235,2,6,3534,136,3879


# Number of Launches per Company

Create a chart that shows the number of space mission launches by organisation.

In [56]:
launches_per_company = df_data.groupby('Organisation').count()['Date'].sort_values(ascending=False)

In [63]:
fig = px.bar(x=launches_per_company.index, 
             y=launches_per_company.values, 
             title='Number of Launches per Company', 
             log_y=True, 
             labels={"x": "Organisation", "y": "Launches"}, 
             color=launches_per_company.values)
fig.update_layout(xaxis_title='Organisation', 
                  yaxis_title='Number of Launches', 
                  coloraxis_showscale=False)
fig.update_xaxes(tickangle = 45)
fig.show()

# Number of Active versus Retired Rockets

How many rockets are active compared to those that are decomissioned? 

In [67]:
rocket_status = df_data.groupby('Rocket_Status').count()['Date']

In [71]:
bar = px.bar(x=['Active', 'Retired'], 
             y=rocket_status, 
             color=rocket_status,
             title='Number of Active vs Retired Rockets')
bar.update_layout(xaxis_title='', 
                  yaxis_title='Number of Rockets', 
                  coloraxis_showscale=False)
bar.show()

# Distribution of Mission Status

How many missions were successful?
How many missions failed?

In [72]:
mission_status = df_data.groupby('Mission_Status').count()['Date']

In [73]:
mission_status

Mission_Status
Failure               339
Partial Failure       102
Prelaunch Failure       4
Success              3879
Name: Date, dtype: int64

In [ ]:
bar = px.bar(x=['Active', 'Retired'], 
             y=rocket_status, 
             color=rocket_status,
             title='Number of Active vs Retired Rockets')
bar.update_layout(xaxis_title='', 
                  yaxis_title='Number of Rockets', 
                  coloraxis_showscale=False)
bar.show()

# How Expensive are the Launches? 

Create a histogram and visualise the distribution. The price column is given in USD millions (careful of missing values). 

# Use a Choropleth Map to Show the Number of Launches by Country

* Create a choropleth map using [the plotly documentation](https://plotly.com/python/choropleth-maps/)
* Experiment with [plotly's available colours](https://plotly.com/python/builtin-colorscales/). I quite like the sequential colour `matter` on this map. 
* You'll need to extract a `country` feature as well as change the country names that no longer exist.

Wrangle the Country Names

You'll need to use a 3 letter country code for each country. You might have to change some country names.

* Russia is the Russian Federation
* New Mexico should be USA
* Yellow Sea refers to China
* Shahrud Missile Test Site should be Iran
* Pacific Missile Range Facility should be USA
* Barents Sea should be Russian Federation
* Gran Canaria should be USA


You can use the iso3166 package to convert the country names to Alpha3 format.

# Use a Choropleth Map to Show the Number of Failures by Country


# Create a Plotly Sunburst Chart of the countries, organisations, and mission status. 

# Analyse the Total Amount of Money Spent by Organisation on Space Missions

# Analyse the Amount of Money Spent by Organisation per Launch

# Chart the Number of Launches per Year

# Chart the Number of Launches Month-on-Month until the Present

Which month has seen the highest number of launches in all time? Superimpose a rolling average on the month on month time series chart. 

# Launches per Month: Which months are most popular and least popular for launches?

Some months have better weather than others. Which time of year seems to be best for space missions?

# How has the Launch Price varied Over Time? 

Create a line chart that shows the average price of rocket launches over time. 

# Chart the Number of Launches over Time by the Top 10 Organisations. 

How has the dominance of launches changed over time between the different players? 

# Cold War Space Race: USA vs USSR

The cold war lasted from the start of the dataset up until 1991. 

## Create a Plotly Pie Chart comparing the total number of launches of the USSR and the USA

Hint: Remember to include former Soviet Republics like Kazakhstan when analysing the total number of launches. 

## Create a Chart that Shows the Total Number of Launches Year-On-Year by the Two Superpowers

## Chart the Total Number of Mission Failures Year on Year.

## Chart the Percentage of Failures over Time

Did failures go up or down over time? Did the countries get better at minimising risk and improving their chances of success over time? 

# For Every Year Show which Country was in the Lead in terms of Total Number of Launches up to and including including 2020)

Do the results change if we only look at the number of successful launches? 

# Create a Year-on-Year Chart Showing the Organisation Doing the Most Number of Launches

Which organisation was dominant in the 1970s and 1980s? Which organisation was dominant in 2018, 2019 and 2020? 